In [1]:
import torch
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Required for 3D plotting

# Load the model
model_path = "/home/maria/LuckyMouse2/saved_models/fold_0/model.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Make sure your model definition is available
model = AttentionNullModel(
    vit_dim=768,              # Adjust to your ViT embedding size
    neuron_embed_dim=32,      # Match the value used in training
    num_neurons=1000,         # Adjust to match your dataset
    attention_dim=64          # Match your model
).to(device)

model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# Extract embeddings
with torch.no_grad():
    neuron_embeddings = model.neuron_embedding.weight.cpu().numpy()  # shape (num_neurons, neuron_embed_dim)

# PCA to 3D
pca = PCA(n_components=3)
neuron_embeddings_3d = pca.fit_transform(neuron_embeddings)

# Plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(
    neuron_embeddings_3d[:, 0],
    neuron_embeddings_3d[:, 1],
    neuron_embeddings_3d[:, 2],
    c='blue',
    alpha=0.7
)

ax.set_title("Neuron Task Embeddings (3D PCA)")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'torch'

In [2]:
!pip install torch

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai